# Miniproyecto 3

### David Soto 17551 & Miguel Valle 17102

In [1]:
### Librerias a utilizar para la simulacion
import numpy as np
import random

### Función generadora de tiempos de estadía/salida

In [2]:
# Funcion para calcular el proximo tiempo de llegada
def tiempoProximaSalida(t, lamda_max, homo):
    if homo:
        return t - ((1/lamda_max)*np.log(random.random()))
    else:
        while True:
            u = random.random()
            t = t - ((1/lamda_max)*np.log(u))
            u = random.random()
            if u <= np.random.poisson(t)/lamda_max:
                return t

### Funcion simuladora de eventos

In [3]:
### Funcion que determina cómo procede el evento segun el tiempo dela simulacion
def evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed):
    ### Si la proxima peticion es el ultimo registrado en el sistema, entonces hay un servidor libre que lo puede atender
    if ta == min(ta,min(td)): 
        t = ta
        Na = Na + 1
        ta = tiempoProximaSalida(t, lamda_max, True)
        A[Na] = t
        ### Si la cantidad de peticiones en el sistema es menor a la cantidad de cpus
        ### es porque la proxima peticion puede ser procesada por un servidor
        if n < no_cpus:
            for i in range(no_cpus):
                if ids[i] == 0:
                    ids[i] = Na
                    NT = np.append(NT, t - A[Na])
                    td[i] = t - ((1/lamda_max)*np.log(random.random()))
                    busyTime[i] = busyTime[i] + td[i] - t
                    break
        n = n + 1
    else:
        ### Esto ocurre en caso de que no haya un servidor desocupado al momento de llegar
        ti = np.argmin(td)
        t = td[ti]
        C[ti] = C[ti] + 1
        D[ids[ti]] = t
        ### En caso la cantidad de clientes sea menor o igual a la cantidad de servidores
        ### Se libera un CPU
        if n <= no_cpus:
            ### Se elimina a la ultima peticion del ID register
            ids[ti] = 0
            td[ti] = np.infty
        ### Si todavia hay mas clientes por atender
        else: 
            indice = max(ids) + 1
            ### Se actualiza el indice
            ids[ti] = indice
            NT = np.append(NT , t - A[indice])
            td[ti] = t - ((1/lamda_max)*np.log(random.random()))
            busyTime[ti] = busyTime[ti] + td[ti] - t
        n = n - 1
        
    ### Retornamos las variables para actualizacion y seguir el proceso de la simulacion 
    return t, n, Na, C, busyTime, A, D, NT, ta, td, ids

### Funcion simuladora del procesamiento de peticiones

In [4]:
### Funcion para realizar la simulacion
def simular(duracion_simulacion, lamda_max, speed,no_cpus):
    ### Variable de tiempo de simulacion
    t = 0
    
    # Variables contadores
    Na = 0
    C = np.zeros(no_cpus) 
    busyTime = np.zeros(no_cpus)
    
    ### Variable estado del sistema
    n = 0

    ### Variables para el control de datos
    A = {}
    D = {}
    NT = []
    
    # Variables de eventos de tiempos de llegada y salida
    ta = tiempoProximaSalida(t,lamda_max, True)
    td = np.zeros(no_cpus) + np.infty
    ids = np.zeros(no_cpus)
    
    # Se realiza la simulacion hasta que se llegue al tiempo final de esta
    while t < duracion_simulacion:
        ### Realizamos la simulacion de un evento al timepo t
        t, n, Na, C, busyTime, A, D, NT, ta, td, ids = evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed)
        
    ### Obtenemos el tiempo de salida de la ultima solicitud atendida
    copiaArreglo = td
    tiemposSalida = []
    for i in copiaArreglo:
        if not np.isinf(i):
            tiemposSalida.append(i)

    ### Obtenemos los segundos promedio por solicitud
    copiaArreglo = NT
    segundoSolicitud = []
    cantidadSolicitudes = 0
    for i in copiaArreglo:
        if i != 0:
            segundoSolicitud.append(1/i)
        cantidadSolicitudes = cantidadSolicitudes + 1
    

    ### Mostramos los resultados de las variables de la somulacion
    print("Solicitudes atendidas: ", C)
    print("Tiempos ocupado de servidores: ", busyTime)
    print("Tiempos desocupado o IDLE de servidores: ", np.maximum(np.ones(no_cpus)*duracion_simulacion - busyTime,0))
    print("Tiempo total de solicitudes en cola: ", np.round(sum(NT),5))
    print("Promedio de solicitudes por segunda en cola:", np.round(np.mean(NT),5))
    print("Promedio de segundos por solicitud en cola:", np.round(sum(segundoSolicitud)/cantidadSolicitudes,5))
    print("Tiempo de salida de la ultima solicitud atendida: ", np.round(max(tiemposSalida),8))

## Task 1 - Simulacion con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto -  Gorilla Megacomputing

In [5]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [143918.]
Tiempos ocupado de servidores:  [59.94026455]
Tiempos desocupado o IDLE de servidores:  [0.05973545]
Tiempo total de solicitudes en cola:  15863.22223
Promedio de solicitudes por segunda en cola: 0.11022
Promedio de segundos por solicitud en cola: 54.69334
Tiempo de salida de la ultima solicitud atendida:  60.00028296


### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [6]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [7.2127e+04 4.3081e+04 1.9636e+04 6.7160e+03 1.7540e+03 3.6100e+02
 6.3000e+01 5.0000e+00 1.0000e+00 0.0000e+00]
Tiempos ocupado de servidores:  [2.99268714e+01 1.79713045e+01 8.16795051e+00 2.81015470e+00
 7.25127103e-01 1.51632744e-01 2.98618998e-02 1.96342784e-03
 1.08215352e-03 0.00000000e+00]
Tiempos desocupado o IDLE de servidores:  [30.07312861 42.02869546 51.83204949 57.1898453  59.2748729  59.84836726
 59.9701381  59.99803657 59.99891785 60.        ]
Tiempo total de solicitudes en cola:  0.0
Promedio de solicitudes por segunda en cola: 0.0
Promedio de segundos por solicitud en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  60.00092545


## Task 2 - Determinacion de numero de servidores para no tener cola con Ants con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [8]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 15

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [7.1702e+04 4.3109e+04 1.9967e+04 6.8150e+03 1.7660e+03 3.7700e+02
 6.3000e+01 9.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00]
Tiempos ocupado de servidores:  [3.00380965e+01 1.80597613e+01 8.34791988e+00 2.84562611e+00
 7.36129184e-01 1.58007001e-01 2.78199361e-02 4.93053320e-03
 7.09323321e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Tiempos desocupado o IDLE de servidores:  [29.96190346 41.94023869 51.65208012 57.15437389 59.26387082 59.841993
 59.97218006 59.99506947 59.99992907 60.         60.         60.
 60.         60.         60.        ]
Tiempo total de solicitudes en cola:  0.0
Promedio de solicitudes por segunda en cola: 0.0
Promedio de segundos por solicitud en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  60.00058492


## Task 3 - Simulacion con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Gorilla Megacomputing

In [9]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [359270.]
Tiempos ocupado de servidores:  [59.76352667]
Tiempos desocupado o IDLE de servidores:  [0.23647333]
Tiempo total de solicitudes en cola:  12593.11293
Promedio de solicitudes por segunda en cola: 0.03505
Promedio de segundos por solicitud en cola: 294.09999
Tiempo de salida de la ultima solicitud atendida:  60.00018072


### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [10]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [1.79905e+05 1.07950e+05 4.97590e+04 1.72110e+04 4.47400e+03 9.21000e+02
 1.48000e+02 2.10000e+01 2.00000e+00 0.00000e+00]
Tiempos ocupado de servidores:  [3.00160521e+01 1.80501638e+01 8.34809200e+00 2.89112194e+00
 7.49681033e-01 1.50164439e-01 2.54940979e-02 3.29356908e-03
 8.75826152e-05 0.00000000e+00]
Tiempos desocupado o IDLE de servidores:  [29.98394788 41.94983618 51.651908   57.10887806 59.25031897 59.84983556
 59.9745059  59.99670643 59.99991242 60.        ]
Tiempo total de solicitudes en cola:  0.0
Promedio de solicitudes por segunda en cola: 0.0
Promedio de segundos por solicitud en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  60.00029036


## Task 4 - Determinacion de numero de servidores para no tener cola con Ants con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [11]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 30

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [1.79717e+05 1.07856e+05 4.91590e+04 1.68730e+04 4.37500e+03 9.12000e+02
 1.57000e+02 2.80000e+01 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00]
Tiempos ocupado de servidores:  [2.99466444e+01 1.79392650e+01 8.22535139e+00 2.81115318e+00
 7.20692020e-01 1.57193417e-01 2.84998396e-02 4.25854813e-03
 1.78034469e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Tiempos desocupado o IDLE de s

## Task 5 - Recomendacion a la junta directiva

Dado que se observa una mejora clara en el tiempo de espera para las solicitudes hechas, se recomienda alquilar 15 servidores para un numero de solicitudes de 2400, y de alquilar 30 servidores para 6000 solicitudes, ya que aunque alquilar más servidores podría reducir aún más el tiempo de espera, no se estaran utilizando todos cuando no se tenga una cantidad de solicitudes muy elevada, por lo cual sería un desperdicio del presupuesto. Es preferible tener menos servidores, pero que todos se estén aprovechando durante un flujo normal de solicitudes realizada, tomando como línea base 2400 y límite superior valores cercanos a 6000. Por lo tanto la cantidad de servidores que se recomienda alquilar se encuentra en un rango entre 15 y 30 servidores.